In [2]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import os


plt.style.use('fivethirtyeight')
script_dir = os.path.abspath('')
data_dir = f"{script_dir}/data/" + "{ticker}.csv"
data_dir

2024-05-21 09:32:58.369868: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-21 09:32:58.370235: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 09:32:58.375599: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 09:32:58.419266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 09:32:59.147051: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

'/home/eris29/APCSA/CSA_AI_Flask_Backend/lstm/data/{ticker}.csv'

In [3]:
class StockDataFrame:
    def __init__(self, ticker):
        self.__ticker = ticker
        self.__data_dir = data_dir
        self.__ticker_data_path = data_dir.format(ticker=self.__ticker.upper())
        self.__stock_df = None
    
    def generate_data_frame(self):
        self.__stock_df = pd.read_csv(self.__ticker_data_path)
    
    def get_data_frame(self):
        return self.__stock_df
    
    def get_ticker(self):
        return self.__ticker

# Create a StockDataFrame instance for AAPL and generate the DataFrame
AAPLDF = StockDataFrame("AAPL")
AAPLDF.generate_data_frame()
aapl_df = AAPLDF.get_data_frame()
aapl_df



,Date,Symbol,Open,Close,Low,High,Volume
0,2014-05-21,AAPL,21.565357,21.653929,21.502144,21.667856,196859600
1,2014-05-22,AAPL,21.664286,21.688213,21.575001,21.780357,200760000
2,2014-05-23,AAPL,21.687500,21.933214,21.659643,21.954643,232209600
3,2014-05-27,AAPL,21.995714,22.343929,21.986786,22.352142,348866000
4,2014-05-28,AAPL,22.357857,22.286072,22.277857,22.493929,315481600
...,...,...,...,...,...,...,...
2512,2024-05-14,AAPL,187.509995,187.429993,186.289993,188.300003,52393600
2513,2024-05-15,AAPL,187.910004,189.720001,187.369995,190.649994,70400000
2514,2024-05-16,AAPL,190.470001,189.839996,189.660004,191.100006,52845200
2515,2024-05-17,AAPL,189.509995,189.869995,189.179993,190.809998,41282900


In [4]:
aapl_df.shape

(2517, 7)

In [13]:
class LSTM:
    def __init__(self, ticker, stock_data):
        self.__ticker = ticker
        self.__stock_data = stock_data
        self.__scaler = MinMaxScaler(feature_range=(0,1))

    def plot_data(self):
        plt.figure(figsize=(16,8))
        plt.title(self.__ticker + ": Close Price History")
        plt.plot(self.__stock_data['Close'])    
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Close price (US)', fontsize=18)
        plt.show()
    
    def scale_split_data(self):
        # Get close data
        close_data = self.__stock_data.filter(['Close'])
        close_dataset = close_data.values
        training_data_len = math.ceil(len(close_dataset) * .8)

        # Scale the data
        scaled_close_data = self.__scaler.fit_transform(close_dataset)
        training_data = scaled_close_data[0:training_data_len, :]

        # Split to Xtrain and Ytrain
        x_train = []
        y_train = []

        for i in range(60, len(training_data)):
            x_train.append(training_data[i-60:i, 0])
            y_train.append(training_data[i, 0])

        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        data = {"x_train" : x_train, "y_train" : y_train}
        return data

        
        
AAPL_obj = LSTM("AAPL", aapl_df)
print(AAPL_obj.scale_split_data())


{'x_train': array([[[0.00000000e+00],
        [1.94295335e-04],
        [1.58274764e-03],
        ...,
        [1.32529956e-02],
        [1.50523055e-02],
        [1.54206722e-02]],

       [[1.94295335e-04],
        [1.58274764e-03],
        [3.91032469e-03],
        ...,
        [1.50523055e-02],
        [1.54206722e-02],
        [1.61007329e-02]],

       [[1.58274764e-03],
        [3.91032469e-03],
        [3.58243847e-03],
        ...,
        [1.54206722e-02],
        [1.61007329e-02],
        [1.77725375e-02]],

       ...,

       [[8.34349731e-01],
        [8.25395651e-01],
        [8.08507619e-01],
        ...,
        [7.07519271e-01],
        [6.85190753e-01],
        [7.10976225e-01]],

       [[8.25395651e-01],
        [8.08507619e-01],
        [7.84422305e-01],
        ...,
        [6.85190753e-01],
        [7.10976225e-01],
        [7.02078785e-01]],

       [[8.08507619e-01],
        [7.84422305e-01],
        [7.99553539e-01],
        ...,
        [7.10976225e-01],
   

In [18]:
class StockDataFrame:
    def __init__(self, ticker):
        self.__ticker = ticker
        self.__data_dir = data_dir
        self.__ticker_data_path = data_dir.format(ticker=self.__ticker.upper())
        self.__stock_df = None
    
    def generate_data_frame(self):
        self.__stock_df = pd.read_csv(self.__ticker_data_path)
    
    def get_data_frame(self):
        return self.__stock_df
    
    def get_ticker(self):
        return self.__ticker

# Create a StockDataFrame instance for AAPL and generate the DataFrame
AAPLDF = StockDataFrame("AAPL")
AAPLDF.generate_data_frame()
aapl_df = AAPLDF.get_data_frame()
aapl_df



,Date,Symbol,Open,Close,Low,High,Volume
0,2014-05-21,AAPL,21.565357,21.653929,21.502144,21.667856,196859600
1,2014-05-22,AAPL,21.664286,21.688213,21.575001,21.780357,200760000
2,2014-05-23,AAPL,21.687500,21.933214,21.659643,21.954643,232209600
3,2014-05-27,AAPL,21.995714,22.343929,21.986786,22.352142,348866000
4,2014-05-28,AAPL,22.357857,22.286072,22.277857,22.493929,315481600
...,...,...,...,...,...,...,...
2512,2024-05-14,AAPL,187.509995,187.429993,186.289993,188.300003,52393600
2513,2024-05-15,AAPL,187.910004,189.720001,187.369995,190.649994,70400000
2514,2024-05-16,AAPL,190.470001,189.839996,189.660004,191.100006,52845200
2515,2024-05-17,AAPL,189.509995,189.869995,189.179993,190.809998,41282900


In [20]:
class LSTM:
    def __init__(self, ticker, stock_data):
        self.__ticker = ticker
        self.__stock_data = stock_data

    def plot_data(self):
        plt.figure(figsize=(16,8))
        plt.title(self.__ticker + ": Close Price History")
        plt.plot(self.__stock_data['Close'])    
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Close price (US)', fontsize=18)
        plt.show()
    
    def scale_data(self):
        close_data = self.__stock_data.filter(['Close'])
        close_dataset = close_data.values
        training_data_len = math.ceil(len(close_dataset) * .8)
        print(training_data_len)

AAPL_obj = LSTM("AAPL", aapl_df)
AAPL_obj.scale_data()


2014
